# Modeling

## Import Requisite Libraries

In [ ]:
######################## Standard Library Imports ##############################
import pandas as pd
import numpy as np
import os
import sys

from eda_toolkit import ensure_directory, generate_table1

######################## Modeling Library Imports ##############################
import shap
from model_tuner.pickleObjects import loadObjects
import model_tuner
import eda_toolkit
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt


# Add the parent directory to sys.path to access 'functions.py'
sys.path.append(os.path.join(os.pardir))

print(
    f"This project uses: \n \n Python {sys.version.split()[0]} \n model_tuner "
    f"{model_tuner.__version__} \n eda_toolkit {eda_toolkit.__version__}"
)

## Set Paths & Read in the Data

In [ ]:
# Define your base paths
# `base_path`` represents the parent directory of your current working directory
base_path = os.path.join(os.pardir)
# Go up one level from 'notebooks' to the parent directory, then into the 'data' folder

data_path = os.path.join(os.pardir, "data")
image_path_png = os.path.join(base_path, "images", "png_images", "modeling")
image_path_svg = os.path.join(base_path, "images", "svg_images", "modeling")

# Use the function to ensure the 'data' directory exists
ensure_directory(data_path)
ensure_directory(image_path_png)
ensure_directory(image_path_svg)

In [ ]:
data_path = "../data/processed/"
model_path = "../mlruns/models/"

In [ ]:
df = pd.read_parquet(os.path.join(data_path, "X.parquet"))  # Change delimiter as needed
df.head()

In [ ]:
df.columns.to_list()

In [ ]:
# create bins for age along with labels such that age as a continuous series
# can be converted to something more manageable for visualization and analysis
bin_ages = [18, 30, 40, 50, 60, 70, 80, 90, 100]
label_ages = [
    "18-29",
    "30-39",
    "40-49",
    "50-59",
    "60-69",
    "70-79",
    "80-89",
    "90-99",
]

df["age_group"] = pd.cut(
    df["Age_years"],
    bins=bin_ages,
    labels=label_ages,
    right=False,  # <-- include left edge, exclude right
    include_lowest=True,  # <-- include the lowest value (e.g. 18)
)

In [ ]:
df

In [ ]:
df_rename = df.copy()

In [ ]:
df_rename = df_rename.rename(
    columns={
        "Intraoperative_Blood_Loss_ml": "Intraoperative Blood Loss",
        "age_group": "Age Group",
        "Surgical_Time_min": "Surgical Time (min)",
        "Intraop_Mean_Heart_Rate_bpm": "Intraoperative Mean Heart Rate (BPM)",
    }
)

In [ ]:
from eda_toolkit import box_violin_plot

metrics_list = [
    "BMI",
    "Intraoperative Blood Loss",
    "Intraoperative Mean Heart Rate (BPM)",
    "Surgical Time (min)",
]
metrics_boxplot_comp = ["Age Group"]
metrics_comp = ["Age Group"]

box_violin_plot(
    df=df_rename,
    metrics_list=metrics_list,
    metrics_comp=metrics_comp,
    image_path_png=image_path_png,
    image_path_svg=image_path_svg,
    save_plots=True,
    show_plot="grid",
    show_legend=False,
    plot_type="boxplot",
    xlabel_rot=90,
    text_wrap=50,
)

In [ ]:
X = pd.read_parquet(os.path.join(data_path, "X.parquet"))
y = pd.read_parquet(os.path.join(data_path, "y_Bleeding_Edema_Outcome.parquet"))

In [ ]:
df = df.join(y, how="inner", on="patient_id")

In [ ]:
table1_cont = generate_table1(
    df,
    include_types="combined",
    groupby_col="Bleeding_Edema_Outcome",
    value_counts=True,
    # apply_bonferroni=True,
    # apply_bh_fdr=True,
    # use_fisher_exact=True,
    decimal_places=2,
)

In [ ]:
table1_cont

In [ ]:
from graphviz import Digraph

# Create revised STROBE diagram with horizontal surgical modality box
dot = Digraph(comment="Enhanced STROBE Diagram with Modality Highlight", format="png")
dot.attr(rankdir="TB", size="10")

# Nodes
dot.node("A", "Patients Evaluated\n(n = 202)", shape="box")
dot.node("B", "Excluded: Under 18\n(n = 8)", shape="box")
dot.node("C", "Final Study Cohort:\nAdult Males ≥ 18\n(n = 194)", shape="box")
dot.node(
    "D",
    "Data Preprocessing\n• Feature engineering\n• Comorbidity filtering\n• No missing data",
    shape="box",
)
dot.node(
    "D2", "Surgical Modality\n• Traditional (n = 132)\n• Laser (n = 62)", shape="box"
)
dot.node(
    "E",
    "Modeling and Evaluation\n• LR, RF, SVM\n• 10-fold CV\n• Balanced class weights",
    shape="box",
)
dot.node(
    "F",
    "Model Calibration\n• Platt scaling\n• Threshold tuning (β = 1, 2)",
    shape="box",
)
dot.node(
    "G",
    "Primary Outcome\nBleeding, Edema, Pain, or Infection\nwithin 7 days",
    shape="box",
)
dot.node("H", "Final Sample Used for Modeling\n(n = 194, 100%)", shape="box")

# Edges (one at a time due to prior error)
dot.edge("A", "B")
dot.edge("B", "C")
dot.edge("C", "D")
dot.edge("D", "D2")
dot.edge("D2", "E")
dot.edge("E", "F")
dot.edge("F", "G")
dot.edge("G", "H")

# Render diagram
dot.render("strobe_modality_emphasized_final", view=True)

In [ ]:
table1_cont.rename(
    columns={
        "1 (n = 62)": "Laser Circumcision (n=62)",
        "0 (n = 132)": "Traditional Circumcision (n=132)",
    },
    inplace=True,
)

In [ ]:
table1_cont

In [ ]:
table1_cont.to_clipboard()

In [ ]:
y.value_counts()

In [ ]:
X.columns.to_list()

In [ ]:
model_lr = loadObjects(
    os.path.join(
        model_path,
        "./849533074301396881/3aae4d2c72274d7599395fe33f08621d/artifacts/lr_Bleeding_Edema_Outcome/model.pkl",
    )
)

model_rf = loadObjects(
    os.path.join(
        model_path,
        "./849533074301396881/d5baeda4999a48788a09423c3f9a6ae0/artifacts/rf_Bleeding_Edema_Outcome/model.pkl",
    )
)

model_svm = loadObjects(
    os.path.join(
        model_path,
        "./849533074301396881/e99151a22e1046e2a4bf4a08b562759c/artifacts/svm_Bleeding_Edema_Outcome/model.pkl",
    )
)

In [ ]:
pipelines_or_models = [model_lr, model_rf, model_svm]

# Model titles
model_titles = [
    "Logistic Regression",
    "Random Forest Classifier",
    "Support Vector Machines",
]


thresholds = {
    "Logistic Regression": next(iter(model_lr.threshold.values())),
    "Random Forest Classifier": next(iter(model_rf.threshold.values())),
    "Support Vector Machines": next(iter(model_svm.threshold.values())),
}

In [ ]:
for col in X.columns:
    if col.startswith("BMI_"):
        print(f"Value Counts for column {col}:\n")
        print(X[col].value_counts())
        print("\n")

## Summarize Model Performance

In [ ]:
pipelines_or_models

In [ ]:
from model_metrics import summarize_model_performance

table3 = summarize_model_performance(
    model=pipelines_or_models,
    X=X,
    y=y,
    model_title=model_titles,
    model_threshold=thresholds,
    return_df=True,
)

In [ ]:
table3

In [ ]:
from model_tuner import evaluate_bootstrap_metrics

In [ ]:
model_svm

## SHAP Summary Plot

In [ ]:
# Step 1: Get transformed features using model's preprocessing pipeline
X_transformed = model_svm.get_preprocessing_and_feature_selection_pipeline().transform(
    X
)

# Optional: Sampling for speed (or just use X_transformed if it's small)
sample_size = 100
X_sample = shap.utils.sample(X_transformed, sample_size, random_state=42)

# Step 2: Get final fitted model (SVC in your pipeline)
final_model = model_svm.estimator.named_steps[model_svm.estimator_name]


# Step 3: Define a pred. function that returns only the probability for class 1
def model_predict(X):
    return final_model.predict_proba(X)[:, 1]


# Step 4: Create SHAP explainer
explainer = shap.KernelExplainer(
    model_predict, X_sample, feature_names=model_svm.get_feature_names()
)

# Step 5: Compute SHAP values for the full dataset or sample
shap_values = explainer.shap_values(X_sample)  # can use X_transformed instead

In [ ]:
# Step 6a: SHAP beeswarm plot (default)
shap.summary_plot(
    shap_values,
    X_sample,
    feature_names=model_svm.get_feature_names(),
    show=False,
)

plt.savefig("shap_summary_beeswarm.png", dpi=600)

In [ ]:
# Step 6b: SHAP bar plot (mean |SHAP value| for each feature)
shap.summary_plot(
    shap_values,
    X_sample,
    feature_names=model_svm.get_feature_names(),
    plot_type="bar",
    show=False,
)

plt.savefig("shap_summary_bar.png", dpi=600)

## Plot SVM Decision Boundary

In [ ]:
from circ_milan.project_functions import plot_svm_decision_boundary_2d

plot_svm_decision_boundary_2d(
    model=model_svm,
    X=X,
    y=y,
    feature_pair=("Intraoperative_Blood_Loss_ml", "Surgical_Technique"),
    title="Temp SVC on 2 Features",
)

## Model Evaluation

## Calibration

In [ ]:
# Plot calibration curves in overlay mode
from model_metrics import show_calibration_curve

show_calibration_curve(
    model=pipelines_or_models,
    X=X,
    y=y,
    model_title=model_titles,
    overlay=True,
    title="",
    save_plot=True,
    image_path_png=image_path_png,
    image_path_svg=image_path_svg,
    text_wrap=40,
    curve_kwgs={
        "Logistic Regression": {"color": "blue", "linewidth": 1},
        "Support Vector Machines": {
            "color": "red",
            # "linestyle": "--",
            "linewidth": 1.5,
        },
        "Decision Tree": {
            "color": "lightblue",
            "linestyle": "--",
            "linewidth": 1.5,
        },
    },
    figsize=(8, 6),
    label_fontsize=10,
    tick_fontsize=10,
    bins=10,
    show_brier_score=True,
    grid=False,
    # gridlines=False,
    linestyle_kwgs={"color": "black"},
    dpi=500,
)

## Confusion Matrices

In [ ]:
from model_metrics import show_confusion_matrix

show_confusion_matrix(
    model=pipelines_or_models,
    X=X,
    y=y,
    model_title=model_titles,
    model_threshold=[thresholds],
    # class_labels=["No Pain", "Class 1"],
    cmap="Blues",
    text_wrap=40,
    save_plot=True,
    image_path_png=image_path_png,
    image_path_svg=image_path_svg,
    grid=True,
    n_cols=3,
    n_rows=1,
    figsize=(4, 4),
    show_colorbar=False,
    label_fontsize=14,
    tick_fontsize=12,
    inner_fontsize=12,
    class_report=True,
    # thresholds=thresholds,
    # custom_threshold=0.5,
    # labels=False,
)

## ROC AUC Curves

In [ ]:
from model_metrics import show_roc_curve

# Plot ROC curves
show_roc_curve(
    model=pipelines_or_models,
    X=X,
    y=y,
    overlay=False,
    model_title=model_titles,
    decimal_places=3,
    # n_cols=3,
    # n_rows=1,
    # curve_kwgs={
    #     "Logistic Regression": {"color": "blue", "linewidth": 2},
    #     "SVM": {"color": "red", "linestyle": "--", "linewidth": 1.5},
    # },
    # linestyle_kwgs={"color": "grey", "linestyle": "--"},
    save_plot=True,
    grid=True,
    n_cols=3,
    figsize=(12, 4),
    # label_fontsize=16,
    # tick_fontsize=16,
    image_path_png=image_path_png,
    image_path_svg=image_path_svg,
    # gridlines=False,
)

In [ ]:
show_roc_curve(
    model=pipelines_or_models,
    X=X,
    y=y,
    overlay=True,
    model_title=model_titles,
    title="AUC ROC - All Models",
    curve_kwgs={
        "Logistic Regression": {"color": "blue", "linewidth": 1},
        "Random Forest": {"color": "lightblue", "linewidth": 1},
        "Support Vector Machines": {
            "color": "red",
            "linestyle": "-",
            "linewidth": 2,
        },
    },
    linestyle_kwgs={"color": "grey", "linestyle": "--"},
    save_plot=True,
    grid=False,
    decimal_places=3,
    figsize=(8, 6),
    # gridlines=False,
    label_fontsize=16,
    tick_fontsize=13,
    image_path_png=image_path_png,
    image_path_svg=image_path_svg,
    dpi=500,
)

## Precision-Recall Curves

In [ ]:
from model_metrics import show_pr_curve

# Plot PR curves
show_pr_curve(
    model=pipelines_or_models,
    X=X,
    y=y,
    # x_label="Hello",
    model_title=model_titles,
    decimal_places=3,
    overlay=False,
    grid=True,
    save_plot=True,
    image_path_png=image_path_png,
    image_path_svg=image_path_svg,
    figsize=(12, 4),
    n_cols=3,
    # tick_fontsize=16,
    # label_fontsize=16,
    # grid=True,
    # gridlines=False,
)

In [ ]:
show_pr_curve(
    model=pipelines_or_models,
    X=X,
    y=y,
    overlay=True,
    model_title=model_titles,
    title="Precision-Recall - All Models",
    curve_kwgs={
        "Logistic Regression": {"color": "blue", "linewidth": 1},
        "Random Forest": {"color": "lightblue", "linewidth": 1},
        "Support Vector Machines": {
            "color": "red",
            "linestyle": "-",
            "linewidth": 2,
        },
    },
    save_plot=True,
    grid=False,
    decimal_places=3,
    figsize=(8, 6),
    # gridlines=False,
    label_fontsize=16,
    tick_fontsize=13,
    image_path_png=image_path_png,
    image_path_svg=image_path_svg,
    dpi=500,
)

## Partial Dependence

In [ ]:
# If X is a DataFrame
feature_names = X.columns

# If X is a NumPy array
feature_names = [f"feature_{i}" for i in range(X.shape[1])]

In [ ]:
from model_metrics import plot_2d_pdp

plot_2d_pdp(
    model=model_svm.estimator,
    X_train=X,  # Ensure this is the corrected DataFrame with valid feature names
    feature_names=X.columns.to_list(),  # Use the aligned feature names
    features=[
        "Age_years",
        "BMI",
        # "Intraoperative_Blood_Loss_ml",
        "Intraop_Mean_Heart_Rate_bpm",
        "Intraop_Mean_Pulse_Ox_Percent",
        ("Surgical_Technique", "Age_years"),
    ],
    title="PDP of house value on CA non-location features",
    grid_figsize=(14, 10),
    individual_figsize=(12, 4),
    label_fontsize=14,
    tick_fontsize=12,
    text_wrap=120,
    plot_type="grid",
    image_path_png=image_path_png,
    save_plots="grid",
)

In [ ]:
model_svm.get_feature_names()

In [ ]:
from model_metrics import plot_3d_pdp

plot_3d_pdp(
    model=model_svm.estimator,
    dataframe=X,
    feature_names=["Surgical_Time_min", "Surgical_Technique"],
    # x_label="House Age",
    # y_label="Average Occupancy",
    # z_label="Partial Dependence",
    title="3D Partial Dependence Plot of Surgical Time in Minutes vs. Surgical Technique",
    html_file_path=image_path_png,
    # image_filename="3d_pdp",
    save_plots="html",
    html_file_name="3d_pdp.html",
    plot_type="static",
    text_wrap=80,
    figsize=(10, 10),
    zoom_out_factor=1.1,
    image_path_png=image_path_png,
    image_path_svg=image_path_svg,
    # grid_resolution=30,
    # label_fontsize=8,
    # tick_fontsize=6,
    # title_x=0.38,
    # top_margin=10,
    # right_margin=50,
    # left_margin=50,
    # cbar_x=0.9,
    # cbar_thickness=25,
    # show_modebar=False,
    # enable_zoom=True,
)

In [ ]:
df.columns

In [ ]:
from model_metrics import plot_3d_pdp

plot_3d_pdp(
    model=model_svm.estimator,
    dataframe=X,
    feature_names=["Surgical_Time_min", "Surgical_Technique"],
    # x_label="House Age",
    # y_label="Average Occupancy",
    # z_label="Partial Dependence",
    title="3D Partial Dependence Plot of House Age vs. Average Occupancy",
    html_file_path=image_path_png,
    # image_filename="3d_pdp",
    html_file_name="3d_pdp.html",
    plot_type="interactive",
    text_wrap=80,
    zoom_out_factor=1.1,
    # image_path_png=image_path_png,
    # image_path_svg=image_path_svg,
    # grid_resolution=30,
    # label_fontsize=8,
    # tick_fontsize=6,
    # title_x=0.38,
    # top_margin=10,
    # right_margin=50,
    # left_margin=50,
    # cbar_x=0.9,
    # cbar_thickness=25,
    # show_modebar=False,
    # enable_zoom=True,
)